In [2]:
import pandas as pd
import nltk
import sys
from nltk.corpus import stopwords

In [3]:
data_train = pd.read_csv('data_train.csv', encoding='latin-1')

In [4]:
#REMOVE STOPWORDS
stop_words = set(stopwords.words('english')) 

In [5]:
# define rejected stopwords bc it has sentiment info
rejected_sw = ["aren't", "couldn", "couldn't", "didn", "didn't", "doesn", "doesn't", "don't", "hadn", "hadn't", "hasn", "hasn't", "haven't", "isn", "isn't", "mightn", "mightn't", "mustn", "mustn't", "needn", "needn't", "no", "nor", "not", "shan't", "shouldn", "shouldn't", "wasn", "wasn't", "weren't", "won't", "wouldn", "wouldn't",  ]

In [6]:
acc_stopwords = []
for i, value in enumerate(stop_words):
    if value not in rejected_sw:
        acc_stopwords.append(value)

In [7]:
# remove stopwords
filtered_sentences = []
for i,words in enumerate(data_train['reviews.text']):
    filtered = [word for word in words.split() if word not in acc_stopwords]
    filtered_sentences.append(' '.join(filtered))

In [8]:
# add new column to store the sentences which stopwords has been removed
data_train['filtered_reviews'] = pd.Series(filtered_sentences, index=data_train.index)
data_train

,Unnamed: 0,reviews.rating,reviews.text,reviews.sentiment,filtered_reviews
0,0,5.0,This product so far has not disappointed. My c...,positive,This product far not disappointed. My children...
1,1,5.0,great for beginner or experienced person. Boug...,positive,great beginner experienced person. Bought gift...
2,2,5.0,Inexpensive tablet for him to use and learn on...,positive,"Inexpensive tablet use learn on, step NABI. He..."
3,3,4.0,I've had my Fire HD 8 two weeks now and I love...,positive,I've Fire HD 8 two weeks I love it. This table...
4,4,5.0,I bought this for my grand daughter when she c...,positive,I bought grand daughter comes visit. I set use...
5,5,5.0,This amazon fire 8 inch tablet is the perfect ...,positive,This amazon fire 8 inch tablet perfect size. I...
6,6,4.0,"Great for e-reading on the go, nice and light ...",positive,"Great e-reading go, nice light weight, price p..."
7,7,5.0,"I gave this as a Christmas gift to my inlaws, ...",positive,"I gave Christmas gift inlaws, husband uncle. T..."
8,8,5.0,Great as a device to read books. I like that i...,positive,Great device read books. I like links borrowed...
9,9,5.0,I love ordering books and reading them with th...,positive,I love ordering books reading reader.


In [ ]:
#data_train.to_csv('filtered_train.csv', index=False)

In [9]:
data_test = pd.read_csv('data_test.csv',  encoding='latin-1')

In [10]:
# remove stopwords
filtered_sentences_test = []
for i,words in enumerate(data_test['reviews.text']):
    filtered = [word for word in words.split() if word not in acc_stopwords]
    filtered_sentences_test.append(' '.join(filtered))

In [11]:
# add new column to store the sentences which stopwords has been removed
data_test['filtered_reviews'] = pd.Series(filtered_sentences_test, index=data_test.index)
data_test.shape

(11887, 4)

In [ ]:
#data_test.to_csv('filtered_test.csv', index=False)

In [ ]:
data_test_w_sent = pd.read_csv('data_test_with_sentiment.csv',  encoding='latin-1')

In [ ]:
# remove stopwords
filtered_sentences_test_sent = []
for i,words in enumerate(data_test_w_sent['reviews.text']):
    filtered = [word for word in words.split() if word not in acc_stopwords]
    filtered_sentences_test_sent.append(' '.join(filtered))

In [ ]:
# add new column to store the sentences which stopwords has been removed
data_test_w_sent['filtered_reviews'] = pd.Series(filtered_sentences_test_sent, index=data_test_w_sent.index)

In [ ]:
#data_test_w_sent.to_csv('filtered_test_w_sentiment.csv', index=False)

In [ ]:
# STEMMING (skip dulu)
from nltk.stem.snowball import SnowballStemmer

In [ ]:
stemmer = SnowballStemmer("english")

In [ ]:
stemmed_train = []
for words in data_train['filtered_reviews']:
    #store each words that have been stemmed in an array of stems
    stems = []
    for word in words.split():
        stem = stemmer.stem(word)
        stems.append(stem)
    #store each sentences from stems in an array of stemmed_train
    stemmed_train.append(' '.join(stems))
stemmed_train

In [ ]:
data_train['stemmed_reviews'] = pd.Series(stemmed_train, index=data_train.index)

In [ ]:
#data_train.to_csv('stem_train.csv', index=False)

In [ ]:
stemmed_test = []
for words in data_test['filtered_reviews']:
    #store each words that have been stemmed in an array of stems
    stems = []
    for word in words.split():
        stem = stemmer.stem(word)
        stems.append(stem)
    #store each sentences from stems in an array of stemmed_train
    stemmed_test.append(' '.join(stems))
stemmed_test

In [ ]:
data_test['stemmed_reviews'] = pd.Series(stemmed_test, index=data_test.index)

In [ ]:
#data_test.to_csv('stem_test.csv', index=False)

In [ ]:
stemmed_test_sent = []
for words in data_test_w_sent['filtered_reviews']:
    #store each words that have been stemmed in an array of stems
    stems = []
    for word in words.split():
        stem = stemmer.stem(word)
        stems.append(stem)
    #store each sentences from stems in an array of stemmed_train
    stemmed_test_sent.append(' '.join(stems))
stemmed_test_sent

In [ ]:
data_test_w_sent['stemmed_reviews'] = pd.Series(stemmed_test_sent, index=data_test_w_sent.index)

In [ ]:
#data_test_w_sent.to_csv('stem_test_w_sent.csv', index=False)
# END SKIP

In [12]:
# binary sentiment. 0 for - 1 for +
bin_sent = []
for i in range(len(data_train)):
    bs = []
    if data_train['reviews.sentiment'][i] == 'negative':
        bin_sent.append(0)
    else:
        bin_sent.append(1)

In [13]:
# store binary sentiments to df
data_train['binary_sentiment'] = pd.Series(bin_sent, index=data_train.index)

In [14]:
#sampling bc data isn't balanced
train_sample = data_train.loc[data_train['reviews.sentiment'] == 'negative'].sample(n=1942, random_state=1234)

In [15]:
train_sample_positive = data_train.loc[data_train['reviews.sentiment'] == 'positive'].sample(n=1942, random_state=1234)
train_sample = train_sample.append(train_sample_positive)
train_sample.shape

(3884, 6)

In [16]:
# sampling the test data as well
test = data_test.sample(n=3884, random_state=1234)

In [17]:
# FEATURE EXTRACTION 1ST APPROACH TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer

In [18]:
vectorizer = TfidfVectorizer()

In [29]:
X = vectorizer.fit_transform(train_sample['filtered_reviews'])

In [21]:
feature_names = vectorizer.get_feature_names()
for col in X.nonzero()[1]:
    print (feature_names[col], ' - ', X[0, col])

need  -  0.2344532887730727
computer  -  0.3120530601599191
savvy  -  0.37597317695345317
wound  -  0.4528480556244011
much  -  0.20169865064377396
helpful  -  0.31661345079657266
came  -  0.3065030817791938
simple  -  0.2773894237604047
basic  -  0.26182834423953855
instructions  -  0.3476461066353581
good  -  0.0
one  -  0.0
liked  -  0.0
it  -  0.0
even  -  0.0
kids  -  0.0
enjoy  -  0.0
this  -  0.0
use  -  0.0
games  -  0.0
much  -  0.20169865064377396
it  -  0.0
bought  -  0.0
mom  -  0.0
the  -  0.0
touchscreen  -  0.0
not  -  0.0
responsive  -  0.0
ipad  -  0.0
camera  -  0.0
little  -  0.0
underpowered  -  0.0
too  -  0.0
didn  -  0.0
think  -  0.0
would  -  0.0
matter  -  0.0
pretty  -  0.0
pixelated  -  0.0
serves  -  0.0
purposes  -  0.0
knew  -  0.0
wasn  -  0.0
high  -  0.0
end  -  0.0
device  -  0.0
going  -  0.0
in  -  0.0
she  -  0.0
able  -  0.0
fumigate  -  0.0
mostly  -  0.0
that  -  0.0
bonus  -  0.0
this  -  0.0
bought  -  0.0
not  -  0.0
birthday  -  0.0
gift  - 

new  -  0.0
but  -  0.0
cheap  -  0.0
stocked  -  0.0
firmware  -  0.0
root  -  0.0
put  -  0.0
custom  -  0.0
after  -  0.0
longer  -  0.0
annoying  -  0.0
pop  -  0.0
adware  -  0.0
feel  -  0.0
table  -  0.0
leat  -  0.0
times  -  0.0
paid  -  0.0
computer  -  0.3120530601599191
one  -  0.0
kids  -  0.0
not  -  0.0
would  -  0.0
like  -  0.0
amazon  -  0.0
received  -  0.0
lot  -  0.0
they  -  0.0
great  -  0.0
tablet  -  0.0
seeking  -  0.0
could  -  0.0
recommend  -  0.0
movies  -  0.0
connect  -  0.0
netflix  -  0.0
books  -  0.0
purchased  -  0.0
grandchildren  -  0.0
ages  -  0.0
power  -  0.0
performance  -  0.0
downloaded  -  0.0
viewed  -  0.0
downfall  -  0.0
grandson  -  0.0
mip  -  0.0
robot  -  0.0
christmas  -  0.0
large  -  0.0
functional  -  0.0
tablets  -  0.0
samsung  -  0.0
dell  -  0.0
others  -  0.0
good  -  0.0
it  -  0.0
this  -  0.0
games  -  0.0
lot  -  0.0
tablet  -  0.0
freezes  -  0.0
using  -  0.0
started  -  0.0
working  -  0.0
freezing  -  0.0
playing  

bought  -  0.0
not  -  0.0
responsive  -  0.0
like  -  0.0
screen  -  0.0
wanted  -  0.0
could  -  0.0
product  -  0.0
read  -  0.0
no  -  0.0
hard  -  0.0
load  -  0.0
books  -  0.0
fine  -  0.0
reading  -  0.0
traveling  -  0.0
tablets  -  0.0
return  -  0.0
screens  -  0.0
light  -  0.0
weight  -  0.0
paper  -  0.0
pdf  -  0.0
file  -  0.0
pdfs  -  0.0
font  -  0.0
enlargable  -  0.0
plus  -  0.0
ended  -  0.0
stuff  -  0.0
available  -  0.0
lots  -  0.0
performance  -  0.0
add  -  0.0
enhance  -  0.0
one  -  0.0
not  -  0.0
best  -  0.0
buy  -  0.0
they  -  0.0
purchase  -  0.0
love  -  0.0
product  -  0.0
me  -  0.0
off  -  0.0
50  -  0.0
two  -  0.0
won  -  0.0
80  -  0.0
online  -  0.0
happy  -  0.0
sales  -  0.0
18  -  0.0
days  -  0.0
today  -  0.0
23  -  0.0
help  -  0.0
15  -  0.0
told  -  0.0
it  -  0.0
the  -  0.0
not  -  0.0
too  -  0.0
high  -  0.0
that  -  0.0
on  -  0.0
like  -  0.0
screen  -  0.0
amazon  -  0.0
them  -  0.0
open  -  0.0
really  -  0.0
wanted  -  0.0
v

devices  -  0.0
stars  -  0.0
decided  -  0.0
saw  -  0.0
slow  -  0.0
still  -  0.0
lights  -  0.0
maybe  -  0.0
actually  -  0.0
turn  -  0.0
many  -  0.0
books  -  0.0
page  -  0.0
an  -  0.0
reader  -  0.0
take  -  0.0
except  -  0.0
motion  -  0.0
work  -  0.0
reading  -  0.0
bit  -  0.0
never  -  0.0
web  -  0.0
others  -  0.0
play  -  0.0
next  -  0.0
every  -  0.0
gets  -  0.0
deal  -  0.0
damaged  -  0.0
disposable  -  0.0
almost  -  0.0
paper  -  0.0
plus  -  0.0
top  -  0.0
order  -  0.0
okay  -  0.0
apple  -  0.0
conditions  -  0.0
sunlight  -  0.0
search  -  0.0
etc  -  0.0
library  -  0.0
function  -  0.0
nearly  -  0.0
see  -  0.0
scroll  -  0.0
quickly  -  0.0
versus  -  0.0
swipe  -  0.0
fifth  -  0.0
refreshes  -  0.0
entirely  -  0.0
slowly  -  0.0
wait  -  0.0
outdoors  -  0.0
reviewers  -  0.0
indicate  -  0.0
sun  -  0.0
glare  -  0.0
pages  -  0.0
with  -  0.0
although  -  0.0
least  -  0.0
suffers  -  0.0
shadow  -  0.0
along  -  0.0
bottom  -  0.0
explanation  

recognize  -  0.0
who  -  0.0
ordering  -  0.0
gimmicks  -  0.0
wear  -  0.0
routinely  -  0.0
soon  -  0.0
robots  -  0.0
dishes  -  0.0
laundry  -  0.0
cook  -  0.0
humans  -  0.0
doing  -  0.0
hopefully  -  0.0
unfold  -  0.0
lifetime  -  0.0
brunsch  -  0.0
good  -  0.0
kids  -  0.0
not  -  0.0
ok  -  0.0
quality  -  0.0
with  -  0.0
around  -  0.0
mess  -  0.0
good  -  0.0
little  -  0.0
like  -  0.0
kindle  -  0.0
really  -  0.0
tablet  -  0.0
ago  -  0.0
so  -  0.0
fire  -  0.0
heard  -  0.0
things  -  0.0
years  -  0.0
cheap  -  0.0
happy  -  0.0
hoping  -  0.0
worried  -  0.0
bang  -  0.0
buck  -  0.0
hope  -  0.0
it  -  0.0
this  -  0.0
use  -  0.0
that  -  0.0
google  -  0.0
youtube  -  0.0
nothing  -  0.0
alexa  -  0.0
lights  -  0.0
turn  -  0.0
home  -  0.0
two  -  0.0
you  -  0.0
okay  -  0.0
had  -  0.0
dot  -  0.0
homes  -  0.0
prefer  -  0.0
rhere  -  0.0
differences  -  0.0
extraordinary  -  0.0
the  -  0.0
not  -  0.0
like  -  0.0
really  -  0.0
offers  -  0.0
free 

...

truly  -  0.0
follow  -  0.0
consistent  -  0.0
refinement  -  0.0
not  -  0.0
works  -  0.0
kindle  -  0.0
purchase  -  0.0
great  -  0.0
20  -  0.0
included  -  0.0
dollars  -  0.0
not  -  0.0
ipad  -  0.0
amazon  -  0.0
get  -  0.0
free  -  0.0
child  -  0.0
apps  -  0.0
purchased  -  0.0
mainly  -  0.0
quality  -  0.0
gets  -  0.0
access  -  0.0
slower  -  0.0
looking  -  0.0
job  -  0.0
done  -  0.0
tell  -  0.0
uses  -  0.0
normally  -  0.0
undergrond  -  0.0
price  -  0.0
kindle  -  0.0
for  -  0.0
wanted  -  0.0
ve  -  0.0
free  -  0.0
sale  -  0.0
model  -  0.0
many  -  0.0
books  -  0.0
everything  -  0.0
especially  -  0.0
reading  -  0.0
downloaded  -  0.0
type  -  0.0
couldn  -  0.0
research  -  0.0
quite  -  0.0
while  -  0.0
have  -  0.0
history  -  0.0
hoped  -  0.0
pass  -  0.0
non  -  0.0
fiction  -  0.0
genealological  -  0.0
amazon  -  0.0
tablet  -  0.0
love  -  0.0
fire  -  0.0
needed  -  0.0
do  -  0.0
exactly  -  0.0
use  -  0.0
the  -  0.0
great  -  0.0
often  

playing  -  0.0
music  -  0.0
happy  -  0.0
useful  -  0.0
weather  -  0.0
speaker  -  0.0
news  -  0.0
89  -  0.0
coming  -  0.0
gadget  -  0.0
nest  -  0.0
wemo  -  0.0
hoped  -  0.0
sonos  -  0.0
integration  -  0.0
hype  -  0.0
2017  -  0.0
integrated  -  0.0
instantaneously  -  0.0
allegedly  -  0.0
much  -  0.20169865064377396
it  -  0.0
didn  -  0.0
would  -  0.0
like  -  0.0
give  -  0.0
great  -  0.0
tablet  -  0.0
love  -  0.0
outside  -  0.0
battery  -  0.0
read  -  0.0
know  -  0.0
no  -  0.0
decided  -  0.0
life  -  0.0
trying  -  0.0
problem  -  0.0
sunlight  -  0.0
night  -  0.0
shot  -  0.0
one  -  0.0
use  -  0.0
would  -  0.0
kindle  -  0.0
love  -  0.0
recommend  -  0.0
ve  -  0.0
no  -  0.0
never  -  0.0
definitely  -  0.0
easy  -  0.0
glare  -  0.0
casual  -  0.0
anywhere  -  0.0
readers  -  0.0
avid  -  0.0
bring  -  0.0
use  -  0.0
apps  -  0.0
middle  -  0.0
work  -  0.0
required  -  0.0
school  -  0.0
educational  -  0.0
elementary  -  0.0
students  -  0.0
it  

love  -  0.0
me  -  0.0
husband  -  0.0
my  -  0.0
alexa  -  0.0
amazing  -  0.0
home  -  0.0
everything  -  0.0
knows  -  0.0
couple  -  0.0
returned  -  0.0
impressed  -  0.0
person  -  0.0
etc  -  0.0
quickly  -  0.0
whether  -  0.0
her  -  0.0
answer  -  0.0
question  -  0.0
brothers  -  0.0
history  -  0.0
ordered  -  0.0
played  -  0.0
math  -  0.0
smartest  -  0.0
event  -  0.0
greeting  -  0.0
it  -  0.0
use  -  0.0
she  -  0.0
go  -  0.0
loves  -  0.0
year  -  0.0
can  -  0.0
money  -  0.0
wrong  -  0.0
old  -  0.0
purchased  -  0.0
you  -  0.0
easy  -  0.0
use  -  0.0
bought  -  0.0
enough  -  0.0
year  -  0.0
old  -  0.0
grandson  -  0.0
easy  -  0.0
four  -  0.0
durable  -  0.0
good  -  0.0
it  -  0.0
gift  -  0.0
best  -  0.0
buy  -  0.0
price  -  0.0
tablet  -  0.0
love  -  0.0
got  -  0.0
gave  -  0.0
purchased  -  0.0
christmas  -  0.0
black  -  0.0
friday  -  0.0
extra  -  0.0
99  -  0.0
thanks  -  0.0
early  -  0.0
39  -  0.0
girlfriend  -  0.0
need  -  0.234453288773

loads  -  0.0
short  -  0.0
order  -  0.0
sunlight  -  0.0
contrast  -  0.0
pays  -  0.0
excellent  -  0.0
backlight  -  0.0
choice  -  0.0
libraries  -  0.0
inside  -  0.0
municipal  -  0.0
effectively  -  0.0
amazon  -  0.0
they  -  0.0
version  -  0.0
thought  -  0.0
limited  -  0.0
android  -  0.0
apps  -  0.0
re  -  0.0
only  -  0.0
negative  -  0.0
one  -  0.0
it  -  0.0
camera  -  0.0
would  -  0.0
mostly  -  0.0
buy  -  0.0
amazon  -  0.0
price  -  0.0
anyone  -  0.0
great  -  0.0
tablet  -  0.0
could  -  0.0
recommend  -  0.0
well  -  0.0
connect  -  0.0
apps  -  0.0
many  -  0.0
want  -  0.0
trying  -  0.0
music  -  0.0
perfect  -  0.0
play  -  0.0
found  -  0.0
speakers  -  0.0
small  -  0.0
ever  -  0.0
looking  -  0.0
all  -  0.0
capacities  -  0.0
the  -  0.0
device  -  0.0
works  -  0.0
price  -  0.0
great  -  0.0
tablet  -  0.0
time  -  0.0
app  -  0.0
well  -  0.0
ok  -  0.0
cable  -  0.0
books  -  0.0
reading  -  0.0
browsing  -  0.0
tv  -  0.0
expectations  -  0.0
ex

loves  -  0.0
price  -  0.0
year  -  0.0
tablet  -  0.0
house  -  0.0
watch  -  0.0
videos  -  0.0
got  -  0.0
free  -  0.0
full  -  0.0
well  -  0.0
wouldn  -  0.0
50  -  0.0
old  -  0.0
tablets  -  0.0
music  -  0.0
play  -  0.0
black  -  0.0
friday  -  0.0
15  -  0.0
to  -  0.0
average  -  0.0
35  -  0.0
listen  -  0.0
mine  -  0.0
practically  -  0.0
knock  -  0.0
swore  -  0.0
suprise  -  0.0
it  -  0.0
kids  -  0.0
screen  -  0.0
love  -  0.0
watch  -  0.0
read  -  0.0
nice  -  0.0
movies  -  0.0
easy  -  0.0
size  -  0.0
she  -  0.0
amazon  -  0.0
really  -  0.0
love  -  0.0
find  -  0.0
alexa  -  0.0
keep  -  0.0
want  -  0.0
work  -  0.0
using  -  0.0
music  -  0.0
asking  -  0.0
awesome  -  0.0
easy  -  0.0
setup  -  0.0
with  -  0.0
song  -  0.0
understands  -  0.0
practice  -  0.0
couch  -  0.0
sentences  -  0.0
good  -  0.0
use  -  0.0
better  -  0.0
bit  -  0.0
but  -  0.0
people  -  0.0
everyday  -  0.0
fits  -  0.0
experienced  -  0.0
one  -  0.0
this  -  0.0
best  -  0

In [ ]:
##2nd Approach of FEATURE EXTRACTION USING CountVectorizer

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv = CountVectorizer(binary=False)

In [ ]:
cv.fit(data_train['filtered_reviews'])

In [ ]:
X = cv.transform(data_train['filtered_reviews'])
print(X)

In [22]:
# TRAINING
from sklearn.linear_model import LogisticRegression

In [23]:
import sklearn.metrics

In [24]:
from sklearn.metrics import accuracy_score

In [25]:
from sklearn.model_selection import train_test_split

In [36]:
validation_size = 0.30 
seed = 1234 #generate same sample
Y = train_sample['binary_sentiment']
X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)

In [37]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [38]:
y_train_pred = logreg.predict(X_train)
y_val_pred = logreg.predict(X_validation)

In [39]:
print("Akurasi Data Training : " + str(logreg.score(X_train, Y_train)))
print("Akurasi Data Testing : " + str(logreg.score(X_validation, Y_validation)))
print("Confusion Matriks Data Training :\n" + str(sklearn.metrics.confusion_matrix(Y_train, y_train_pred)))

Akurasi Data Training : 0.88373804267844
Akurasi Data Testing : 0.7847341337907375
Confusion Matriks Data Training :
[[1220  151]
 [ 165 1182]]


In [40]:
print("Confusion Matriks Data Testing :\n" + str(sklearn.metrics.confusion_matrix(Y_validation, y_val_pred)))

Confusion Matriks Data Testing :
[[450 121]
 [130 465]]


In [47]:
import pickle

In [48]:
saved_model = pickle.dumps(logreg)
saved_model

b'\x80\x03csklearn.linear_model.logistic\nLogisticRegression\nq\x00)\x81q\x01}q\x02(X\x07\x00\x00\x00penaltyq\x03X\x02\x00\x00\x00l2q\x04X\x04\x00\x00\x00dualq\x05\x89X\x03\x00\x00\x00tolq\x06G?\x1a6\xe2\xeb\x1cC-X\x01\x00\x00\x00Cq\x07G?\xf0\x00\x00\x00\x00\x00\x00X\r\x00\x00\x00fit_interceptq\x08\x88X\x11\x00\x00\x00intercept_scalingq\tK\x01X\x0c\x00\x00\x00class_weightq\nNX\x0c\x00\x00\x00random_stateq\x0bNX\x06\x00\x00\x00solverq\x0cX\t\x00\x00\x00liblinearq\rX\x08\x00\x00\x00max_iterq\x0eKdX\x0b\x00\x00\x00multi_classq\x0fX\x03\x00\x00\x00ovrq\x10X\x07\x00\x00\x00verboseq\x11K\x00X\n\x00\x00\x00warm_startq\x12\x89X\x06\x00\x00\x00n_jobsq\x13K\x01X\x08\x00\x00\x00classes_q\x14cnumpy.core.multiarray\n_reconstruct\nq\x15cnumpy\nndarray\nq\x16K\x00\x85q\x17C\x01bq\x18\x87q\x19Rq\x1a(K\x01K\x02\x85q\x1bcnumpy\ndtype\nq\x1cX\x02\x00\x00\x00i8q\x1dK\x00K\x01\x87q\x1eRq\x1f(K\x03X\x01\x00\x00\x00<q NNNJ\xff\xff\xff\xffJ\xff\xff\xff\xffK\x00tq!b\x89C\x10\x00\x00\x00\x00\x00\x00\x00\x00\x01

In [41]:
for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, Y_train)
    print ("Accuracy for C=%s: %s" % (c, accuracy_score(Y_validation, lr.predict(X_validation))))
    
final_wc = LogisticRegression(C=0.05)
final_wc.fit(X, Y)
print ("Final Accuracy: %s" % accuracy_score(Y, final_wc.predict(X_test)))


Accuracy for C=0.01: 0.7495711835334476
Accuracy for C=0.05: 0.7692967409948542
Accuracy for C=0.25: 0.7813036020583191
Accuracy for C=0.5: 0.7864493996569468
Accuracy for C=1: 0.7847341337907375
Final Accuracy: 0.4945932028836251


In [45]:
X_test = vectorizer.fit_transform(test['filtered_reviews'])

In [46]:
feature_names = vectorizer.get_feature_names()
for col in X_test.nonzero()[1]:
    print (feature_names[col], ' - ', X_test[0, col])

we  -  0.16464714802108685
used  -  0.372858890505731
ipads  -  0.3030337028312088
house  -  0.19251802783237507
this  -  0.11881327396040617
taken  -  0.2975049846126915
learning  -  0.21768278367895202
get  -  0.14703121266430724
slightly  -  0.28826597501916046
different  -  0.21844250927796693
format  -  0.29262830953077423
works  -  0.13400779854904857
great  -  0.09235096180214775
the  -  0.117924304245825
price  -  0.15737808456494917
point  -  0.2520181694446237
perfect  -  0.18435881962759162
kids  -  0.17532227441530396
use  -  0.10994921405438596
without  -  0.19419113964840104
worry  -  0.2714781449358321
this  -  0.11881327396040617
get  -  0.14703121266430724
the  -  0.117924304245825
present  -  0.0
husband  -  0.0
originally  -  0.0
asked  -  0.0
radio  -  0.0
home  -  0.0
office  -  0.0
he  -  0.0
set  -  0.0
relatively  -  0.0
quick  -  0.0
amazed  -  0.0
sound  -  0.0
quality  -  0.0
loved  -  0.0
versatility  -  0.0
could  -  0.0
specific  -  0.0
station  -  0.0
pla

bought  -  0.0
nice  -  0.0
pretty  -  0.0
features  -  0.0
prime  -  0.0
cool  -  0.0
whim  -  0.0
day  -  0.0
pricing  -  0.0
gets  -  0.0
updated  -  0.0
often  -  0.0
this  -  0.11881327396040617
great  -  0.09235096180214775
one  -  0.0
best  -  0.0
market  -  0.0
watch  -  0.0
this  -  0.11881327396040617
well  -  0.0
looking  -  0.0
device  -  0.0
fast  -  0.0
slick  -  0.0
connects  -  0.0
network  -  0.0
easily  -  0.0
job  -  0.0
really  -  0.0
it  -  0.0
love  -  0.0
not  -  0.0
one  -  0.0
still  -  0.0
fire  -  0.0
much  -  0.0
hd  -  0.0
if  -  0.0
awesome  -  0.0
like  -  0.0
night  -  0.0
see  -  0.0
haven  -  0.0
found  -  0.0
first  -  0.0
size  -  0.0
feels  -  0.0
brightness  -  0.0
reading  -  0.0
think  -  0.0
kindle  -  0.0
kindles  -  0.0
kicking  -  0.0
saw  -  0.0
store  -  0.0
user  -  0.0
friendly  -  0.0
offers  -  0.0
many  -  0.0
options  -  0.0
functions  -  0.0
have  -  0.0
mention  -  0.0
dwarfs  -  0.0
last  -  0.0
make  -  0.0
complaint  -  0.0
auto 

space  -  0.0
she  -  0.0
camper  -  0.0
life  -  0.0
use  -  0.10994921405438596
love  -  0.0
still  -  0.0
device  -  0.0
alexa  -  0.0
thing  -  0.0
night  -  0.0
don  -  0.0
always  -  0.0
every  -  0.0
fall  -  0.0
asleep  -  0.0
although  -  0.0
äôt  -  0.0
understand  -  0.0
great  -  0.09235096180214775
use  -  0.10994921405438596
bought  -  0.0
time  -  0.0
bedroom  -  0.0
use  -  0.10994921405438596
he  -  0.0
it  -  0.0
easy  -  0.0
something  -  0.0
purchased  -  0.0
really  -  0.0
dad  -  0.0
70  -  0.0
needed  -  0.0
enjoys  -  0.0
the  -  0.117924304245825
without  -  0.19419113964840104
it  -  0.0
another  -  0.0
speaker  -  0.0
able  -  0.0
right  -  0.0
play  -  0.0
music  -  0.0
convenient  -  0.0
time  -  0.0
need  -  0.0
also  -  0.0
touch  -  0.0
echo  -  0.0
plus  -  0.0
won  -  0.0
powerful  -  0.0
response  -  0.0
whatever  -  0.0
answer  -  0.0
itself  -  0.0
equip  -  0.0
bluetooth  -  0.0
pair  -  0.0
ask  -  0.0
request  -  0.0
anything  -  0.0
acceptable  

ll  -  0.0
good  -  0.0
cool  -  0.0
month  -  0.0
need  -  0.0
place  -  0.0
ve  -  0.0
battery  -  0.0
speak  -  0.0
hard  -  0.0
far  -  0.0
recognized  -  0.0
correctly  -  0.0
orders  -  0.0
feel  -  0.0
con  -  0.0
cover  -  0.0
off  -  0.0
it  -  0.0
little  -  0.0
bought  -  0.0
do  -  0.0
games  -  0.0
loves  -  0.0
daughter  -  0.0
limited  -  0.0
watching  -  0.0
she  -  0.0
understand  -  0.0
young  -  0.0
cartoons  -  0.0
it  -  0.0
not  -  0.0
speed  -  0.0
package  -  0.0
bought  -  0.0
pretty  -  0.0
fast  -  0.0
charger  -  0.0
charges  -  0.0
that  -  0.0
anything  -  0.0
find  -  0.0
said  -  0.0
normal  -  0.0
disappointing  -  0.0
sure  -  0.0
wrong  -  0.0
seems  -  0.0
self  -  0.0
explanatory  -  0.0
price  -  0.15737808456494917
use  -  0.10994921405438596
it  -  0.0
year  -  0.0
things  -  0.0
easy  -  0.0
old  -  0.0
gift  -  0.0
pretty  -  0.0
absolutely  -  0.0
loves  -  0.0
happen  -  0.0
granddaughter  -  0.0
she  -  0.0
especially  -  0.0
bargain  -  0.0

ve  -  0.0
though  -  0.0
more  -  0.0
got  -  0.0
appletv  -  0.0
from  -  0.0
cast  -  0.0
macbook  -  0.0
gotta  -  0.0
great  -  0.09235096180214775
the  -  0.117924304245825
loved  -  0.0
it  -  0.0
love  -  0.0
one  -  0.0
still  -  0.0
wife  -  0.0
new  -  0.0
much  -  0.0
faster  -  0.0
old  -  0.0
like  -  0.0
generation  -  0.0
kindle  -  0.0
look  -  0.0
got  -  0.0
made  -  0.0
early  -  0.0
unwilling  -  0.0
update  -  0.0
standing  -  0.0
we  -  0.16464714802108685
one  -  0.0
tv  -  0.0
easy  -  0.0
bought  -  0.0
install  -  0.0
two  -  0.0
works  -  0.13400779854904857
great  -  0.09235096180214775
would  -  0.0
better  -  0.0
cable  -  0.0
recommend  -  0.0
everyone  -  0.0
perfect  -  0.18435881962759162
he  -  0.0
set  -  0.0
it  -  0.0
bought  -  0.0
able  -  0.0
play  -  0.0
loves  -  0.0
music  -  0.0
father  -  0.0
him  -  0.0
reminders  -  0.0
wants  -  0.0
great  -  0.09235096180214775
outstanding  -  0.0
features  -  0.0
friendly  -  0.0
many  -  0.0
battery 

...

control  -  0.0
lights  -  0.0
spotify  -  0.0
via  -  0.0
commands  -  0.0
hub  -  0.0
updates  -  0.0
weather  -  0.0
timers  -  0.0
condo  -  0.0
house  -  0.19251802783237507
get  -  0.14703121266430724
great  -  0.09235096180214775
use  -  0.10994921405438596
home  -  0.0
it  -  0.0
never  -  0.0
amazon  -  0.0
minutes  -  0.0
another  -  0.0
started  -  0.0
device  -  0.0
fire  -  0.0
tv  -  0.0
much  -  0.0
hd  -  0.0
cut  -  0.0
cable  -  0.0
so  -  0.0
easy  -  0.0
roku  -  0.0
like  -  0.0
best  -  0.0
using  -  0.0
but  -  0.0
good  -  0.0
app  -  0.0
haven  -  0.0
family  -  0.0
apps  -  0.0
read  -  0.0
prime  -  0.0
member  -  0.0
month  -  0.0
network  -  0.0
many  -  0.0
all  -  0.0
netflix  -  0.0
sling  -  0.0
hulu  -  0.0
streaming  -  0.0
antenna  -  0.0
with  -  0.0
already  -  0.0
plus  -  0.0
devices  -  0.0
channels  -  0.0
unit  -  0.0
wanted  -  0.0
needed  -  0.0
question  -  0.0
carry  -  0.0
got  -  0.0
saving  -  0.0
connect  -  0.0
find  -  0.0
made  -  0

the  -  0.117924304245825
use  -  0.10994921405438596
it  -  0.0
would  -  0.0
not  -  0.0
fire  -  0.0
bought  -  0.0
day  -  0.0
battery  -  0.0
charge  -  0.0
every  -  0.0
impressed  -  0.0
order  -  0.0
trouble  -  0.0
holding  -  0.0
july  -  0.0
repurchase  -  0.0
sound  -  0.0
etc  -  0.0
not  -  0.0
to  -  0.0
easy  -  0.0
add  -  0.0
list  -  0.0
calendar  -  0.0
good  -  0.0
do  -  0.0
required  -  0.0
features  -  0.0
somewhat  -  0.0
limited  -  0.0
acceptable  -  0.0
setup  -  0.0
reminders  -  0.0
bose  -  0.0
fits  -  0.0
attractive  -  0.0
design  -  0.0
terms  -  0.0
minimum  -  0.0
glossary  -  0.0
décor  -  0.0
use  -  0.10994921405438596
smart  -  0.0
not  -  0.0
tv  -  0.0
very  -  0.0
apple  -  0.0
fast  -  0.0
content  -  0.0
really  -  0.0
user  -  0.0
many  -  0.0
options  -  0.0
mention  -  0.0
simple  -  0.0
streaming  -  0.0
tried  -  0.0
box  -  0.0
tweaking  -  0.0
devices  -  0.0
on  -  0.0
for  -  0.0
doesn  -  0.0
products  -  0.0
two  -  0.0
having  -

ethernet  -  0.0
better  -  0.0
cable  -  0.0
like  -  0.0
plug  -  0.0
connection  -  0.0
used  -  0.372858890505731
it  -  0.0
another  -  0.0
one  -  0.0
variety  -  0.0
kindles  -  0.0
purchase  -  0.0
low  -  0.0
ve  -  0.0
wanted  -  0.0
uses  -  0.0
ones  -  0.0
storage  -  0.0
knew  -  0.0
current  -  0.0
became  -  0.0
great  -  0.09235096180214775
the  -  0.117924304245825
it  -  0.0
amazon  -  0.0
fire  -  0.0
tv  -  0.0
features  -  0.0
needs  -  0.0
worked  -  0.0
liked  -  0.0
we  -  0.16464714802108685
this  -  0.11881327396040617
works  -  0.13400779854904857
great  -  0.09235096180214775
worry  -  0.2714781449358321
set  -  0.0
it  -  0.0
little  -  0.0
amazon  -  0.0
fire  -  0.0
shows  -  0.0
year  -  0.0
tv  -  0.0
tablet  -  0.0
recommend  -  0.0
movies  -  0.0
old  -  0.0
bought  -  0.0
buy  -  0.0
money  -  0.0
favorite  -  0.0
lot  -  0.0
loves  -  0.0
don  -  0.0
daughter  -  0.0
watch  -  0.0
also  -  0.0
everyone  -  0.0
controls  -  0.0
age  -  0.0
downloade

large  -  0.0
easily  -  0.0
kindle  -  0.0
user  -  0.0
friendly  -  0.0
big  -  0.0
enough  -  0.0
it  -  0.0
not  -  0.0
amazon  -  0.0
work  -  0.0
will  -  0.0
ok  -  0.0
controlled  -  0.0
wantedto  -  0.0
great  -  0.09235096180214775
price  -  0.15737808456494917
love  -  0.0
internet  -  0.0
fast  -  0.0
item  -  0.0
streaming  -  0.0
fact  -  0.0
hard  -  0.0
wire  -  0.0
lightning  -  0.0
great  -  0.09235096180214775
use  -  0.10994921405438596
easy  -  0.0
activate  -  0.0
touch  -  0.0
sounds  -  0.0
must  -  0.0
works  -  0.13400779854904857
love  -  0.0
tv  -  0.0
wifi  -  0.0
add  -  0.0
ability  -  0.0
signal  -  0.0
strong  -  0.0
flawlessly  -  0.0
established  -  0.0
acorn  -  0.0
british  -  0.0
productions  -  0.0
great  -  0.09235096180214775
love  -  0.0
speed  -  0.0
way  -  0.0
netflix  -  0.0
load  -  0.0
enjoy  -  0.0
the  -  0.117924304245825
kids  -  0.17532227441530396
home  -  0.0
amazon  -  0.0
bought  -  0.0
gift  -  0.0
best  -  0.0
google  -  0.0
vi

room  -  0.0
this  -  0.11881327396040617
without  -  0.19419113964840104
it  -  0.0
love  -  0.0
fast  -  0.0
convenient  -  0.0
kindle  -  0.0
simple  -  0.0
second  -  0.0
lightweight  -  0.0
fits  -  0.0
down  -  0.0
trips  -  0.0
bag  -  0.0
weighing  -  0.0
used  -  0.372858890505731
this  -  0.11881327396040617
could  -  0.0
well  -  0.0
it  -  0.0
looking  -  0.0
not  -  0.0
amazon  -  0.0
device  -  0.0
fed  -  0.0
fire  -  0.0
internet  -  0.0
thought  -  0.0
tv  -  0.0
port  -  0.0
back  -  0.0
no  -  0.0
might  -  0.0
picture  -  0.0
seen  -  0.0
wifi  -  0.0
remote  -  0.0
best  -  0.0
start  -  0.0
ll  -  0.0
purchased  -  0.0
work  -  0.0
play  -  0.0
issue  -  0.0
automatic  -  0.0
switch  -  0.0
prime  -  0.0
even  -  0.0
expect  -  0.0
netflix  -  0.0
excellent  -  0.0
need  -  0.0
video  -  0.0
load  -  0.0
receiving  -  0.0
days  -  0.0
tried  -  0.0
screen  -  0.0
interface  -  0.0
control  -  0.0
case  -  0.0
returned  -  0.0
though  -  0.0
battery  -  0.0
plus  -

In [49]:
# Load the pickled model
classifier_from_pickle = pickle.loads(saved_model)

# Use the loaded pickled model to make predictions
classifier_from_pickle.predict(X_test)

ValueError: X has 5160 features per sample; expecting 5829

In [50]:
X_test.shape

(3884, 5160)

In [51]:
X.shape

(3884, 5829)